In [40]:
 import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [41]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    # plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    # plt.legend([string, 'val_'+string])
    plt.show()

In [42]:
df1 = pd.read_csv('short_oversampled_CICDDoS2019.csv')
df1.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label,target
0,331784,2383541,25018,172.16.0.5-192.168.50.1-900-51191-17,172.16.0.5,900,192.168.50.1,51191,17,2018-12-01 11:27:48.991419,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_LDAP,1
1,681538,2787990,3686,172.16.0.5-192.168.50.1-900-30666-17,172.16.0.5,900,192.168.50.1,30666,17,2018-12-01 11:29:40.752984,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_LDAP,1
2,129580,9703,7328,192.168.50.8-74.208.236.171-58739-80-6,192.168.50.8,58739,74.208.236.171,80,6,2018-12-01 09:58:53.243688,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,BENIGN,0
3,422212,630396,1480,172.16.0.5-192.168.50.1-780-975-17,172.16.0.5,780,192.168.50.1,975,17,2018-12-01 10:40:27.990422,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_NTP,1
4,67101,5034493,1847,172.217.10.98-192.168.50.7-80-52361-6,192.168.50.7,52361,172.217.10.98,80,6,2018-12-01 13:29:02.216554,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,BENIGN,0


In [43]:
df2 = df1.groupby([' Timestamp'])[' Flow Duration'].count()
df2 = pd.DataFrame(df2).reset_index()
df2.columns=[' Timestamp','BiFlowsCount']
df = df1.merge(df2, left_on=' Timestamp', right_on=' Timestamp')
#df = df.sort_values(' Timestamp')
del df['Unnamed: 0']
del df['Unnamed: 0.1']
del df['Unnamed: 0.1.1']
del df['target']
del df[' Inbound']
df

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Label,BiFlowsCount
0,172.16.0.5-192.168.50.1-900-51191-17,172.16.0.5,900,192.168.50.1,51191,17,2018-12-01 11:27:48.991419,2,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,DrDoS_LDAP,1
1,172.16.0.5-192.168.50.1-900-30666-17,172.16.0.5,900,192.168.50.1,30666,17,2018-12-01 11:29:40.752984,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,DrDoS_LDAP,1
2,192.168.50.8-74.208.236.171-58739-80-6,192.168.50.8,58739,74.208.236.171,80,6,2018-12-01 09:58:53.243688,7286835,2,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,159
3,192.168.50.8-74.208.236.171-58739-80-6,192.168.50.8,58739,74.208.236.171,80,6,2018-12-01 09:58:53.243688,7286835,2,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,159
4,192.168.50.8-74.208.236.171-58739-80-6,192.168.50.8,58739,74.208.236.171,80,6,2018-12-01 09:58:53.243688,7286835,2,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,172.16.0.5-192.168.50.1-783-1285-17,172.16.0.5,783,192.168.50.1,1285,17,2018-12-01 11:25:22.598985,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,DrDoS_LDAP,1
999996,172.16.0.5-192.168.50.1-798-28534-17,172.16.0.5,798,192.168.50.1,28534,17,2018-12-01 11:26:13.214604,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,DrDoS_LDAP,1
999997,172.16.0.5-192.168.50.1-975-62605-17,172.16.0.5,975,192.168.50.1,62605,17,2018-12-01 11:28:10.369171,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,DrDoS_LDAP,1
999998,172.16.0.5-192.168.50.1-518-50242-17,172.16.0.5,518,192.168.50.1,50242,17,2018-12-01 11:23:48.951612,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,DrDoS_LDAP,1


In [44]:
df.columns

Index(['Flow ID', ' Source IP', ' Source Port', ' Destination IP',
       ' Destination Port', ' Protocol', ' Timestamp', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Pa

In [45]:
features = ['BiFlowsCount', ' Source Port', ' Destination Port', ' URG Flag Count', ' ACK Flag Count', ' Fwd Packet Length Min',
' Avg Fwd Segment Size', 'Fwd Packets/s', ' Min Packet Length', ' Fwd Packet Length Mean', ' Flow Packets/s', ' Packet Length Mean',
' CWE Flag Count', ' Protocol', ' Average Packet Size', ' Down/Up Ratio', 'Flow Bytes/s', 'Init_Win_bytes_forward', ' RST Flag Count',
' Bwd IAT Min']

In [72]:
fraction = 0.5
df = df.sample(frac=fraction, replace=True, random_state=1)

In [73]:
def targetify(s):
    if s == 'BENIGN':
        return 0
    else:
        return 1

In [74]:
df['Target']=df[' Label'].apply(targetify)

In [75]:
X = df[features]
X[features] = X[features].apply(pd.to_numeric, errors='coerce', axis=1)

C:\Users\kdagi\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [76]:
y = df['Target']

In [77]:
X = X.fillna(0)

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(400000, 20) (400000,)
(100000, 20) (100000,)


In [79]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_train.values)
X_train = pd.DataFrame(x_scaled,columns=features)
x_scaled_test = min_max_scaler.fit_transform(X_test.values)
X_test = pd.DataFrame(x_scaled_test,columns=features)

In [80]:
model = tf.keras.Sequential([
    # An Embedding layer expecting input vocab of size 100000, and
# output embedding dimension of size 64.
    tf.keras.layers.Embedding(100000, 64), # since it doesn't consider "words," the embedding doesn't really matter
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), # units = 64
    tf.keras.layers.Dense(64, activation='relu'), # 64 units, activation = relu
    tf.keras.layers.Dense(1, activation='sigmoid') # 1 unit, activation sigmoid
    # tf.keras.layers.Dense(1, activation='softmax') # loss too big
]) 

In [81]:
model.compile(loss='binary_crossentropy',
              # optimizer='sgd', # almost same
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

#Once the model is created, you can config the model with losses and metrics with model.compile(), train the model with model.fit(), or use the model to do prediction with model.predict().

In [82]:
history = model.fit(X_train.values, y_train.values, epochs=3)


Train on 400000 samples
Epoch 1/3
400000/400000 [==============================] - 8177s 20ms/sample - loss: 0.4329 - accuracy: 0.7504
Epoch 2/3
400000/400000 [==============================] - 30615s 77ms/sample - loss: 0.4060 - accuracy: 0.7676
Epoch 3/3
400000/400000 [==============================] - 6608s 17ms/sample - loss: 0.4049 - accuracy: 0.7683


In [83]:
pred_class = model.predict(X_test.values[:])
print(pred_class)

[[0.6640037 ]
 [0.99981433]
 [0.6640037 ]
 ...
 [0.0019826 ]
 [0.3901229 ]
 [0.99981433]]


In [84]:
def myRound(x, r):
    if x>r/float(1000):
        return 1
    else:
        return 0

In [85]:
print(round(pred_class[2][0]))
print(round(pred_class[1][0]))

1.0
1.0


In [86]:
predictions = [int(round(x[0])) for x in pred_class]
true_class = list(y_test)

In [87]:
print(true_class[:10], len(true_class))
print(predictions[:10], len(predictions))

[0, 1, 0, 0, 0, 0, 0, 0, 0, 1] 100000
[1, 1, 1, 0, 0, 0, 1, 0, 0, 1] 100000


In [88]:
compdf = pd.DataFrame({'pred_class':predictions, 'true_class':true_class})
compdf = compdf.sort_values('pred_class', ascending=False)
print(str(compdf.head(10)))

       pred_class  true_class
0               1           0
54139           1           1
54072           1           1
54074           1           1
54075           1           1
54080           1           1
54081           1           1
54082           1           1
54084           1           1
54085           1           1


In [89]:
predictions = list(compdf['pred_class'].apply(myRound, r=225))
true_class = list(compdf['true_class'])

In [90]:
print(predictions[:10])
print(true_class[:10])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [91]:
from sklearn.metrics import confusion_matrix

In [92]:
confm = confusion_matrix(true_class, predictions)
confm

array([[40757,  9121],
       [14080, 36042]], dtype=int64)

In [93]:
tn, fp, fn, tp = confm.ravel()
(tn + tp)/(tn+tp+fn+fp)

0.76799

In [94]:
print("False alarm rate is")
fp/(tn+fp)

False alarm rate is


0.1828661935121697

In [95]:
print("Precision is")
Precision = tp/(tp+fp)
Precision

Precision is


0.7980426455284193

In [96]:
print("Recall is")
Recall = tp/(tp+fn)
Recall

Recall is


0.7190854315470253

In [97]:
print("F1 is")
2*Precision*Recall/(Precision+Recall)

F1 is


0.7565094191110878